In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC ## Silver-to-Gold: KPI & Analytics Table Generation

# COMMAND ----------

import pyspark.sql.functions as F

silver_base_path = "/mnt/silver"
gold_base_path = "/mnt/gold"

# COMMAND ----------

# MAGIC %md ### Load Silver Tables

# COMMAND ----------

tables_to_load = [
    "country",
    "league",
    "match",
    "player",
    "player_attributes",
    "team",
    "team_attributes"
]

print("Loading Silver tables...")
for table_name in tables_to_load:
    try:
        df = spark.read.format("delta").load(f"{silver_base_path}/{table_name}")
        df.createOrReplaceTempView(f"{table_name}_silver")
        print(f"Successfully loaded '{table_name}_silver'")
    except Exception as e:
        print(f"Error loading table {table_name}: {e}")

print("All Silver tables loaded into temporary views.")

# COMMAND ----------

# MAGIC %md ### Gold Table 1 - Team Win Rates

# COMMAND ----------

try:
    print("Calculating Gold Table 1: Team Win Rates...")
    
    team_win_rates_df = spark.sql("""
        WITH all_matches AS (
            SELECT 
                home_team_api_id AS team_api_id,
                CASE 
                    WHEN home_team_goal > away_team_goal THEN 'Win'
                    WHEN home_team_goal = away_team_goal THEN 'Draw'
                    ELSE 'Loss'
                END AS result
            FROM match_silver
            
            UNION ALL
            
            SELECT 
                away_team_api_id AS team_api_id,
                CASE 
                    WHEN away_team_goal > home_team_goal THEN 'Win'
                    WHEN away_team_goal = home_team_goal THEN 'Draw'
                    ELSE 'Loss'
                END AS result
            FROM match_silver
        ),
        
        team_stats AS (
            SELECT
                team_api_id,
                COUNT(*) AS total_matches_played,
                SUM(CASE WHEN result = 'Win' THEN 1 ELSE 0 END) AS total_wins,
                SUM(CASE WHEN result = 'Loss' THEN 1 ELSE 0 END) AS total_losses,
                SUM(CASE WHEN result = 'Draw' THEN 1 ELSE 0 END) AS total_draws
            FROM all_matches
            GROUP BY team_api_id
        )

        SELECT 
            t.team_long_name,
            s.total_matches_played,
            s.total_wins,
            s.total_losses,
            s.total_draws,
            ROUND((s.total_wins / s.total_matches_played) * 100, 2) AS win_rate_percent
        FROM team_stats s
        JOIN team_silver t ON s.team_api_id = t.team_api_id
        ORDER BY win_rate_percent DESC
    """)
    
    team_win_rates_df.write.format("delta").mode("overwrite").save(f"{gold_base_path}/team_win_rates")
    
    print("Successfully created Gold table: 'team_win_rates'")
    display(team_win_rates_df)

except Exception as e:
    print(f"Error creating 'team_win_rates': {e}")

# COMMAND ----------

# MAGIC %md ### Gold Table 2 - League Statistics

# COMMAND ----------

try:
    print("Calculating Gold Table 2: League Statistics...")
    
    league_stats_df = spark.sql("""
        SELECT 
            c.name AS country,
            l.name AS league_name,
            COUNT(m.id) AS total_matches,
            SUM(m.home_team_goal + m.away_team_goal) AS total_goals,
            ROUND(AVG(m.home_team_goal + m.away_team_goal), 2) AS avg_goals_per_match,
            ROUND(AVG(m.home_team_goal), 2) AS avg_home_goals,
            ROUND(AVG(m.away_team_goal), 2) AS avg_away_goals
        FROM match_silver m
        JOIN country_silver c ON m.country_id = c.id
        JOIN league_silver l ON m.league_id = l.id
        GROUP BY c.name, l.name
        ORDER BY total_goals DESC
    """)
    
    league_stats_df.write.format("delta").mode("overwrite").save(f"{gold_base_path}/league_stats")
    
    print("Successfully created Gold table: 'league_stats'")
    display(league_stats_df)
    
except Exception as e:
    print(f"Error creating 'league_stats': {e}")

# COMMAND ----------

# MAGIC %md ### Gold Table 3 - Top 20 Rated Players

# COMMAND ----------

try:
    print("Calculating Gold Table 3: Top 20 Rated Players...")

    top_players_df = spark.sql("""
        WITH latest_player_stats AS (
            SELECT 
                player_api_id,
                overall_rating,
                potential,
                preferred_foot,
                attacking_work_rate,
                defensive_work_rate,
                ROW_NUMBER() OVER(
                    PARTITION BY player_api_id 
                    ORDER BY date DESC
                ) as rn
            FROM player_attributes_silver
        )
        
        SELECT 
            p.player_name,
            s.overall_rating,
            s.potential,
            s.preferred_foot,
            s.attacking_work_rate,
            s.defensive_work_rate
        FROM latest_player_stats s
        JOIN player_silver p ON s.player_api_id = p.player_api_id
        WHERE s.rn = 1
        ORDER BY s.overall_rating DESC
        LIMIT 20
    """)

    top_players_df.write.format("delta").mode("overwrite").save(f"{gold_base_path}/top_20_players")
    
    print("Successfully created Gold table: 'top_20_players'")
    display(top_players_df)

except Exception as e:
    print(f"Error creating 'top_20_players': {e}")

# COMMAND ----------

# MAGIC %md ### Verification

# COMMAND ----------

print("Gold layer tables:")
dbutils.fs.ls("/mnt/gold")




In [ ]:
# COMMAND ----------

# MAGIC %md ### Visualization 1: Top 20 Players by Overall Rating

# COMMAND ----------

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid")

top_players_pd = spark.read.format("delta").load(f"{gold_base_path}/top_20_players").toPandas()

plt.figure(figsize=(12, 10))
sns.barplot(
    x="overall_rating", 
    y="player_name", 
    data=top_players_pd, 
    palette="viridis"
)
plt.title("Top 20 Players by Overall Rating", fontsize=16, fontweight='bold')
plt.xlabel("Overall Rating", fontsize=12)
plt.ylabel("Player Name", fontsize=12)
plt.tight_layout()
plt.show()

# COMMAND ----------

# MAGIC %md ### Visualization 2: Team Win Rates (Top 15 Teams)

# COMMAND ----------

team_win_rates_pd = spark.read.format("delta").load(f"{gold_base_path}/team_win_rates").toPandas()

top_15_teams = team_win_rates_pd.head(15)

plt.figure(figsize=(14, 8))
sns.barplot(
    x="win_rate_percent", 
    y="team_long_name", 
    data=top_15_teams, 
    palette="coolwarm"
)
plt.title("Top 15 Teams by Win Rate", fontsize=16, fontweight='bold')
plt.xlabel("Win Rate (%)", fontsize=12)
plt.ylabel("Team Name", fontsize=12)
plt.tight_layout()
plt.show()

# COMMAND ----------

# MAGIC %md ### Visualization 3: Total Matches Played by Top 15 Teams

# COMMAND ----------

plt.figure(figsize=(14, 8))
sns.barplot(
    x="total_matches_played", 
    y="team_long_name", 
    data=top_15_teams, 
    palette="mako"
)
plt.title("Total Matches Played (Top 15 Teams by Win Rate)", fontsize=16, fontweight='bold')
plt.xlabel("Total Matches Played", fontsize=12)
plt.ylabel("Team Name", fontsize=12)
plt.tight_layout()
plt.show()

# COMMAND ----------

# MAGIC %md ### Visualization 4: League Statistics - Total Goals

# COMMAND ----------

league_stats_pd = spark.read.format("delta").load(f"{gold_base_path}/league_stats").toPandas()

plt.figure(figsize=(14, 8))
sns.barplot(
    x="total_goals", 
    y="league_name", 
    data=league_stats_pd, 
    palette="rocket"
)
plt.title("Total Goals by League", fontsize=16, fontweight='bold')
plt.xlabel("Total Goals", fontsize=12)
plt.ylabel("League Name", fontsize=12)
plt.tight_layout()
plt.show()

# COMMAND ----------

# MAGIC %md ### Visualization 5: Average Goals Per Match by League

# COMMAND ----------

plt.figure(figsize=(14, 8))
sns.barplot(
    x="avg_goals_per_match", 
    y="league_name", 
    data=league_stats_pd, 
    palette="flare"
)
plt.title("Average Goals Per Match by League", fontsize=16, fontweight='bold')
plt.xlabel("Average Goals Per Match", fontsize=12)
plt.ylabel("League Name", fontsize=12)
plt.tight_layout()
plt.show()

# COMMAND ----------

# MAGIC %md ### Visualization 6: Home vs Away Goals Comparison

# COMMAND ----------

fig, ax = plt.subplots(figsize=(14, 8))

x_pos = range(len(league_stats_pd))
width = 0.35

ax.barh([i - width/2 for i in x_pos], league_stats_pd['avg_home_goals'], 
        width, label='Home Goals', color='steelblue')
ax.barh([i + width/2 for i in x_pos], league_stats_pd['avg_away_goals'], 
        width, label='Away Goals', color='coral')

ax.set_yticks(x_pos)
ax.set_yticklabels(league_stats_pd['league_name'])
ax.set_xlabel('Average Goals', fontsize=12)
ax.set_ylabel('League Name', fontsize=12)
ax.set_title('Home vs Away Goals by League', fontsize=16, fontweight='bold')
ax.legend()
plt.tight_layout()
plt.show()

# COMMAND ----------

# MAGIC %md ### Visualization 7: Win/Loss/Draw Distribution (Top 10 Teams)

# COMMAND ----------

top_10_teams = team_win_rates_pd.head(10)

fig, ax = plt.subplots(figsize=(14, 8))

x_pos = range(len(top_10_teams))
width = 0.25

ax.barh([i - width for i in x_pos], top_10_teams['total_wins'], 
        width, label='Wins', color='green', alpha=0.8)
ax.barh(x_pos, top_10_teams['total_draws'], 
        width, label='Draws', color='orange', alpha=0.8)
ax.barh([i + width for i in x_pos], top_10_teams['total_losses'], 
        width, label='Losses', color='red', alpha=0.8)

ax.set_yticks(x_pos)
ax.set_yticklabels(top_10_teams['team_long_name'])
ax.set_xlabel('Number of Matches', fontsize=12)
ax.set_ylabel('Team Name', fontsize=12)
ax.set_title('Win/Draw/Loss Distribution (Top 10 Teams)', fontsize=16, fontweight='bold')
ax.legend()
plt.tight_layout()
plt.show()

# COMMAND ----------

# MAGIC %md ### Visualization 8: Player Potential vs Overall Rating

# COMMAND ----------

plt.figure(figsize=(12, 8))
sns.scatterplot(
    x="overall_rating", 
    y="potential", 
    data=top_players_pd,
    s=150,
    color='purple',
    alpha=0.6
)
plt.title("Player Potential vs Overall Rating (Top 20 Players)", fontsize=16, fontweight='bold')
plt.xlabel("Overall Rating", fontsize=12)
plt.ylabel("Potential", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# COMMAND ----------

# MAGIC %md ### Summary Statistics

# COMMAND ----------

print("=== GOLD LAYER SUMMARY ===\n")

print(f"Total teams analyzed: {len(team_win_rates_pd)}")
print(f"Total leagues analyzed: {len(league_stats_pd)}")
print(f"Top 20 players identified\n")

print("Top 3 Teams by Win Rate:")
print(team_win_rates_pd[['team_long_name', 'win_rate_percent']].head(3).to_string(index=False))

print("\nMost Competitive League (by avg goals per match):")
top_league = league_stats_pd.loc[league_stats_pd['avg_goals_per_match'].idxmax()]
print(f"{top_league['league_name']} - {top_league['avg_goals_per_match']} goals/match")

print("\nTop Rated Player:")
print(f"{top_players_pd.iloc[0]['player_name']} - Rating: {top_players_pd.iloc[0]['overall_rating']}")
